## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Python imports

In [ ]:
import glob
import os

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import cv2

from moviepy.editor import VideoFileClip
from IPython.display import HTML

---
## Camera calibration matrix and distortion coefficients

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
obj_points = np.zeros((6*9,3), np.float32)
obj_points[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
object_points = []
image_points = []

# Make a list of calibration images
calibration_images = glob.glob('camera_calibration/calibration*.jpg')

# Step through the list and search for chessboard corners
for image_file_name in calibration_images:
    # Load the image
    image = mpimg.imread(image_file_name)

    # Create a working copy
    working_copy = np.copy(image)
    
    # Convert to grayscale
    gray = cv2.cvtColor(working_copy, cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points and image points
    if ret == True:
        object_points.append(obj_points)
        image_points.append(corners)

# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)

---
## Extract thresholded binary image

In [ ]:
def color_and_gradient_thresholding(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)

    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Stack each channel
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary

---
## Create top-down view

In [ ]:
def perspective_transformation(img):
    height, width = img.shape

    # Define 4 source points src = np.float32([[,],[,],[,],[,]])
    src = np.float32([
        [(width / 2) - 60, height / 2 + 100], # top left
        [(width / 6) - 20, height],           # bottom left
        [(width * 5 / 6) + 60, height],       # bottom right
        [(width / 2) + 70, height / 2 + 100]  # top right
    ])
    
    # Define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    dst = np.float32([
        [(width / 4), 0],                     # top left
        [(width / 4), height],                # bottom left
        [(width * 3 / 4), height],            # bottom right
        [(width * 3 / 4), 0]                  # top right
    ])
    
    # Use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    
    # Use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, (width, height), flags=cv2.INTER_LINEAR)
    
    # Use cv2.getPerspectiveTransform() to get Minv, the transform matrix
    Minv = cv2.getPerspectiveTransform(dst, src)

    return warped, Minv

---
## Find lane lines using sliding windows

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        ### Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(leftx, lefty, rightx, righty, binary_warped):
    ### Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    # Generate array of points to drat the lane
    points_left_side = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    points_right_side = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    points_stacked = np.int_(np.hstack((points_left_side, points_right_side)))

    # Colors in the left and right lane regions
    binary_warped[lefty, leftx] = [255, 0, 0]
    binary_warped[righty, rightx] = [0, 0, 255]
    
    # Draw the lane
    cv2.fillPoly(binary_warped, points_stacked, (0,255,0))

    return left_fit, right_fit, binary_warped

---
## Measure the curvature and offset

In [ ]:
def curvature_and_offset(leftx, lefty, rightx, righty, height, width):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Transform pixel to meters
    leftx = leftx * xm_per_pix
    lefty = lefty * ym_per_pix
    rightx = rightx * xm_per_pix
    righty = righty * ym_per_pix

    # Fit a polynomial
    left_fit_cr = np.polyfit(lefty, leftx, 2)
    right_fit_cr = np.polyfit(righty, rightx, 2)

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = height * ym_per_pix

    ##### Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    curverad = (left_curverad + right_curverad) / 2

    left_point = np.poly1d(left_fit_cr)(y_eval)
    right_point = np.poly1d(right_fit_cr)(y_eval)

    lane_center = (left_point + right_point) / 2
    image_center = (width * xm_per_pix) / 2

    offset = lane_center - image_center
    
    return int(curverad), int(offset)

---
## Line detecting pipeline

In [ ]:
def detect_lines(image, output_file_folder=None):
    # Create a working copy
    working_copy = np.copy(image)
    
    # Undistort the image
    undistorted = cv2.undistort(working_copy, mtx, dist, None, mtx)
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "undistorted.jpg"), undistorted)
        
    # Extract thresholded binary image
    thresholded = color_and_gradient_thresholding(undistorted)
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "thresholded.jpg"), thresholded)
    
    # Create top-down view
    warped, Minv = perspective_transformation(thresholded)
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "warped.jpg"), warped)
    
    # Find lane lines using sliding windows
    leftx, lefty, rightx, righty, warped_stacked = find_lane_pixels(warped)
    left_fit, right_fit, warped_lanes = fit_polynomial(leftx, lefty, rightx, righty, warped_stacked)
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "warped_lanes.jpg"), warped_lanes)

    # Get the curvature and offset
    height, width, _ = warped_lanes.shape
    curverad, offset = curvature_and_offset(leftx, lefty, rightx, righty, height, width)
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "warped_lanes.jpg"), warped_lanes)

    # Warp the image with lanes to original image space
    lanes = cv2.warpPerspective(warped_lanes, Minv, (width, height))
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "lanes.jpg"), lanes)

    # Apply the lanes to undistorted image
    result = cv2.addWeighted(undistorted, 1, lanes, 0.3, 0)
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "result_no_text.jpg"), result)
    
    # Print offset and curvature on the result image
    place_of_center = "right" if offset < 0 else "left"
    curvature_text = "Radius of Curvature = {}(m)".format(curverad)
    offser_text = "Vehicle is {0:.2f}m ".format(abs(offset)) + "{} of center".format(place_of_center)
    cv2.putText(result, curvature_text, (50,60), cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
    cv2.putText(result, offser_text, (50,120), cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
    if output_file_folder is not None:
        mpimg.imsave(os.path.join(output_file_folder, "result.jpg"), result)
    
    return result

---
## Running the pipeline on straight_lines test images

In [ ]:
# Test the pipeline on straight_lines test images
straight_lines_images = glob.glob('test_images/straight_lines*.jpg')

# Step through the list and apply 
for image_file_name in straight_lines_images:
    # Load the image
    image = mpimg.imread(image_file_name)
    
    # Create folder for output files
    output_file_folder = os.path.join('output_images', os.path.basename(image_file_name))
    if not os.path.isdir(output_file_folder):
        os.makedirs(output_file_folder)
    mpimg.imsave(os.path.join(output_file_folder, "base_image.jpg"), image)

    # Detect lines
    result_image = detect_lines(image, output_file_folder)

    # Display results side by side
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(result_image)
    ax2.set_title('Result Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

---
## Running the pipeline on the rest of test images

In [ ]:
# Test the pipeline on test images
test_images = glob.glob('test_images/test*.jpg')

# Step through the list and apply 
for image_file_name in test_images:
    # Load the image
    image = mpimg.imread(image_file_name)

    # Create folder for output files
    output_file_folder = os.path.join('output_images', os.path.basename(image_file_name))
    if not os.path.isdir(output_file_folder):
        os.makedirs(output_file_folder)
    mpimg.imsave(os.path.join(output_file_folder, "base_image.jpg"), image)

    # Detect lines
    result_image = detect_lines(image, output_file_folder)

    # Display results side by side
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(result_image)
    ax2.set_title('Result Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

---
## Running the pipeline on project_video.mp4

In [ ]:
# Load the video
project_video = VideoFileClip("project_video.mp4")
# Process the video
project_video_lines = project_video.fl_image(detect_lines)
# Save the processed video
project_video_lines.write_videofile("project_video_lanes.mp4", audio=False)

In [ ]:
HTML("""
<video width="640" height="360" controls>
  <source src="{0}">
</video>
""".format("project_video_lanes.mp4"))

---
## Running the pipeline on challenge_video.mp4

In [ ]:
# Load the video
project_video = VideoFileClip("challenge_video.mp4")
# Process the video
project_video_lines = project_video.fl_image(detect_lines)
# Save the processed video
project_video_lines.write_videofile("challenge_video_lanes.mp4", audio=False)

In [ ]:
HTML("""
<video width="640" height="360" controls>
  <source src="{0}">
</video>
""".format("challenge_video_lanes.mp4"))

---
## Running the pipeline on harder_challenge_video.mp4

In [ ]:
# Load the video
project_video = VideoFileClip("harder_challenge_video.mp4")
# Process the video
project_video_lines = project_video.fl_image(detect_lines)
# Save the processed video
project_video_lines.write_videofile("harder_challenge_video_lanes.mp4", audio=False)

In [ ]:
HTML("""
<video width="640" height="360" controls>
  <source src="{0}">
</video>
""".format("harder_challenge_video_lanes.mp4"))